## Full preprocessing pipeline of the GeoLife dataset for the GNN Route Recommendation project
author: Ludovico Comito (comito.1837155@studenti.uniroma1.it)

The GeoLife dataset contains GPS a total of 17.621 total trajectories of everyday trips over five years. For each trip, a datapoint is constituted by (latitude, longitude, timestamp). Among these users, 74 of them have also labelled them with the transportation mode they were using (walk, car, bike...). This dataset contains data from various cities, and we will focus on the city of Beijing, being the one that contains most of the data.

For the sake of this project, we are interested only in the portion of data that is annotated, as we want to build a recommender system that takes into account also the desired mean of transport.

Each entry of the final dataset will contain:
- ID of the user
- Path trajectory
- Transportation mode
- Timestamp


## Step 1: get the labelled trajectories from the raw dataset
You can download the raw dataset from [this page](https://www.microsoft.com/en-us/download/details.aspx?id=52367)

In [ ]:
import os
import pandas as pd
from datetime import datetime, timedelta
import pickle
import numpy as np

The Data folder contains one subfolder names with the ID of each user. Within each user folder we have a trajectory folders and (optionally) a labels.txt file, that contains labelling with transport mode.

We are interesting in isolating only labelled trajectories, from which we will store latitude, longitude and timestamp.

The data for each user will be then stored in a temporary csv file for further processing.


In [ ]:
data_dir = 'geolife-data/Data'
dirlist = os.listdir(data_dir)

label_dirs = []
for dir in dirlist[1:]:
  trajlist = os.listdir(data_dir + '/' + dir)
  if 'labels.txt' in trajlist:
    label_dirs.append(data_dir + '/' + dir)

traj_columns = ['latitude','longitude','height','days_total','date','time']

def year_plus(col):
  return col.replace(year=col.year + 1)

def get_trans_trip(record_dt,ref_df):
    time_fit = (record_dt >= ref_df['Start Time']) & (record_dt <= ref_df['End Time'])
    nmatch = time_fit.sum()
    if nmatch == 0:
        t_idx = None
    else:
        if nmatch > 1:
            print('More than one mode match!')
        t_idx = ref_df.loc[time_fit].iloc[0].name
    return t_idx

match_paths = []
for ldirs in label_dirs:
  all_traj = pd.DataFrame()
  print('Processing: ' + ldirs)
  print(ldirs)
  user = ldirs[-3:]
  trajpath = ldirs + '/Trajectory/'
  traj_files = os.listdir(trajpath)
  trip_trans = pd.read_csv(ldirs+'/labels.txt',sep='\t')
  trip_trans['Start Time'] = pd.to_datetime(trip_trans['Start Time'])
  trip_trans['End Time'] = pd.to_datetime(trip_trans['End Time'])

  trip_s_dates = trip_trans['Start Time'].dt.date.unique()
  trip_e_dates = trip_trans['End Time'].dt.date.unique()
  trip_a_dates = np.unique(np.append(trip_s_dates,trip_e_dates))
  for tf in traj_files:
    traj_df = (pd.read_csv(trajpath+tf
                            ,skiprows=6
                            ,usecols=[0,1,3,4,5,6]
                            ,names=traj_columns)
                .assign(record_dt = lambda x: pd.to_datetime(
                                        x['date'] + ' ' + x['time']
                                        ),
                        user = user
                        )
                )
    if traj_df['record_dt'].dt.date.isin(trip_a_dates).any():
        print('matches possible')
        traj_df['trans_trip'] = traj_df.apply(lambda x: get_trans_trip(x.record_dt,trip_trans),axis=1)
        has_trip = ~(traj_df.trans_trip.isnull())
        traj_df['trans_mode'] = np.nan
        traj_df.loc[has_trip,'trans_mode'] = traj_df.loc[has_trip].apply(lambda x: trip_trans.loc[x.trans_trip,'Transportation Mode'],axis=1)
        all_traj = pd.concat([all_traj,traj_df])
  print(f'Saving trajectories for user {user}')
  all_traj.to_csv('csvs/'+f'user_{user}_'+'_trajectories.csv')


## Step 2: isolate single trajectories for each user
Now we have to face a problem: we have successfully separated trajectories for each user, but they are sored as a unique csv from which we have to isolate single trips.

The following function separates single trips based on shifts of datetime of changes in transport mode. Single trips are then pickled and stored.

In [ ]:
# Function to parse date and time and return a datetime object
def parse_datetime(date_str, time_str):
    return datetime.strptime(f"{date_str} {time_str}", "%Y-%m-%d %H:%M:%S")

def filter_and_store(base_dir, allowed_modes=['walk', 'car', 'bus']):
    # Define the allowed transportation modes
    allowed_modes = ['walk', 'car', 'taxi', 'bus']
    csv_dirs = os.listdir(base_dir)

    for user_csv in csv_dirs:
        user_file = 'csvs/' + user_csv
        user_df = pd.read_csv(user_file)
        print(f'Processing {user_file}')
        if len(user_df.columns) > 5:
            user_df = user_df[['latitude','longitude','date','time','user','trans_mode']]
            user_id = user_df['user'][0]
            
            # Extracting trajectories
            trajectories = []
            current_trajectory = []

            prev_mode = None
            prev_datetime = None
            for _, row in user_df.iterrows():
                if row['trans_mode'] not in allowed_modes:
                    # Skip modes not in allowed_modes
                    continue

                current_datetime = parse_datetime(row['date'], row['time'])
                if (prev_datetime is not None and current_datetime - prev_datetime > timedelta(hours=3)) or (prev_mode!=row['trans_mode']):
                    # Check if current trajectory is long enough before adding
                    if len(current_trajectory) >= 10:
                        trajectories.append(current_trajectory)
                    current_trajectory = []
                
                current_trajectory.append(row.tolist())
                prev_mode = row['trans_mode']
                prev_datetime = current_datetime

            # Check the last trajectory
            if current_trajectory and len(current_trajectory) >= 10:
                trajectories.append(current_trajectory)
                
            print(f'Saving pickled_data/array_{user_id}.pkl')
            # Pickling the array
            with open(f'pickled_data/array_{user_id}.pkl', 'wb') as file:
                pickle.dump(trajectories, file)

filter_and_store('csvs')

## Step 3: preprocess single trajectories

Raw GPS trajectories are tipically noisy, so we will preprocess them in the following way:
1. Outlier removal.
2. Datapoints sumsampling (compression).
3. Staypoints and duplicates removal.
4. Map matching.

For outlier removals and compression we will use the skmob library.

After these processing, we will have clean representation of our trips on the map. To make our data usable by the model, we have to apply two other steps:

**Making paths continuous**

Our architecture needs to have continous trajectories, meaning that the node at step k+1 must be a neighbour of node at step k (there needs to be an edge between them). To achieve this, we will simply connect non-neighbouring nodes by computing the shortest path between them using the Networkx library.

**Cutting length**

After preprocessing, our trips are still very long compared to the dataset that was used in the original NeuroMLR paper. A simple data analysis shows that the average length of nodes in a trip for the original dataset is 36, with a standard deviation of 34. In our data we have an average length of 243 with a standard deviation of 787 resulting in a far more challenging setting. To make our dataset comparable, trips are cut to a maximum length of 70.



In [ ]:
from osmnx.distance import nearest_edges
import skmob
from skmob.preprocessing import filtering, compression, detection
import geopandas as gpd
import haversine
from haversine import haversine, Unit
import networkx as nx

In [ ]:
# Load graph, nodes, and edges
G = pickle.load(open('map/graph_with_haversine.pkl', 'rb'))
edge_df = gpd.read_file('mlr_data/edges.shp')
node_df = gpd.read_file('mlr_data/nodes.shp')
a = node_df['osmid'].to_numpy()
b = node_df[['y', 'x']].to_numpy()
map_node_osm_to_coords = {e:(u,v) for e,(u,v) in zip(a,b)} # osmid to coords
del a
del b

map_edge_id_to_u_v = edge_df[['u', 'v']].to_numpy()
map_u_v_to_edge_id = {(u,v):i for i,(u,v) in enumerate(map_edge_id_to_u_v)}
unique_edge_labels = list(map_u_v_to_edge_id.values())		# these are from OSM map data, not train data

In [ ]:
def map_edges_to_ids(edges, map_u_v_to_edge_id):
    return [map_u_v_to_edge_id[(u,v)] for u,v,i in edges]
    
def remove_consecutive_duplicates(numbers):
    # This function removes consecutive duplicates from a list while preserving order
    if not numbers:
        return []

    # Initialize the result list with the first element
    result = [numbers[0]]

    # Iterate over the list, compare each element with the last element in result
    for num in numbers[1:]:
        if num != result[-1]:
            result.append(num)

    return result

def remove_stay_points(data, distance_threshold):
    cleaned_latitudes = []
    cleaned_longitudes = []

    previous_point = (data['Latitude'][0], data['Longitude'][0])
    cleaned_latitudes.append(previous_point[0])
    cleaned_longitudes.append(previous_point[1])

    for lat, lon in zip(data['Latitude'][1:], data['Longitude'][1:]):
        current_point = (lat, lon)
        dist = haversine(previous_point, current_point, unit=Unit.METERS)
        print(f'dist: {dist}m')
        if dist >= distance_threshold:
            cleaned_latitudes.append(lat)
            cleaned_longitudes.append(lon)
            previous_point = current_point

    return {'Latitude': cleaned_latitudes, 'Longitude': cleaned_longitudes}

def build_path(G, edges):
    path = []
    for i in range(len(edges) - 1):
        start_edge = edges[i]
        end_edge = edges[i + 1]
        # Find the shortest path between the nodes of consecutive nearest edges
        try:
            shortest_path = nx.shortest_path(G, source=start_edge[1], target=end_edge[0])
            path.extend(shortest_path)
        except nx.NetworkXNoPath:
            print("No path", start_edge, end_edge)
    return path



In [ ]:
base_dir = 'pickled_data/'
dirs = os.listdir(base_dir)

processed_trajectories = []
for user_file in dirs:
    if user_file == '.DS_Store':
        continue
    print(f'processing: {user_file}')
    user_dir = base_dir + user_file
    with open(user_dir, 'rb') as file:
        user_trajectories = pickle.load(file)
    for trajectory in user_trajectories:
        user_traj_df = pd.DataFrame(trajectory)
        user_traj_df.columns = ['Latitude', 'Longitude', 'Date','Time','User_ID','Transport_mode'] 
        
        # Convert to skmob format
        user_traj_df['datetime'] = pd.to_datetime(user_traj_df['Date'] + ' ' + user_traj_df['Time'])

        # Remove staypoints
        user_traj_df = remove_stay_points(user_traj_df, 5)
        
        # Load skmob
        tdf = skmob.TrajDataFrame(user_traj_df, latitude='Latitude', longitude='Longitude', datetime='datetime',user_id='User_ID')
        # Filter outliers
        ftdf = filtering.filter(tdf, max_speed_kmh=500.)
        # Compress trajectory
        ctdf = compression.compress(ftdf, spatial_radius_km=0.1)
        
        # Map match edges
        edge_ids = nearest_edges(G, ctdf['lng'], ctdf['lat'])
        
        indexed_eges = map_edges_to_ids(edge_ids, map_u_v_to_edge_id)
        indexed_eges = remove_consecutive_duplicates(indexed_eges)

        # Make path continuous
        indexed_edges = build_path(G, indexed_eges)
        
        # Normalize distance
        indexed_edges = indexed_edges[:min(70, len(indexed_edges))]

        # Store data
        final_data = (ctdf['uid'][0], indexed_eges, ctdf['Transport_mode'][0],ctdf['datetime'][0]) # user_id, trajectory_edges, transport_mode, starting datetime
        processed_trajectories.append(final_data)

# save pickled processed trajectories
with open('to_edges/processed_trajectories_all.pkl', 'wb') as file:
    pickle.dump(processed_trajectories, file)
    
    

## Step 4: preparing data for the model
In order to reduce the cost of computation during training, we will preprocess the data in order to obtain a tensor dataset that we can feed straight to the model.

- User ids: this is already numeric data, so we will only have to transform data into tensors.
- Transport modes: we have three transport modes that are Walk, Car and Taxi. In this case we simply encode them into integers 0, 1, 2 using a dict.
- Trajectories: we don't need any further preprocessing.

**Timestamps**

While preprocessing for the other data is pretty straightforward, we need to put attention in the choice of the useful information about timestamps and how to encode it. Raw timestamps are in the format YEAR:MONTH:DAY, HOUR:MINUTES:SECONDS. In my design choices, I decided to just keep the month, day of the week and hour. This is because the year seems less correlated to the choice of the path, as well as the specific minute and second that we are deciding. Instead month can suggest the season we are in, the day of the week can tell us whether we are in a work day or in the weekend and the hour can tell us about different choices of our user when travelling by day or night. These informations seem very meaningful to extract patterns.

There's another choice to be addressed: how to encode time data in a meaningful way. We could simply choose to select the integers that represents months, weekdays and hours. But let's make an example: midnight is represented by the number 12 and one in the morning by number 1. These hours are close, but their numeric representation is bery different. In order to get a better representation, we will use sinusoidal encodings, that exploit the cyclical nature of our data by means of a sine/cosine representation.

In [ ]:
import torch

In [ ]:
transport_mode_dict = {
    'walk': 0,
    'car': 1,
    'bus': 2,
}

def encode_transport_mode(mode, transport_mode_dict):
    return transport_mode_dict[mode]

def cyclical_encode(data, max_val):
    sin_encoded = torch.sin(2 * np.pi * data / max_val)
    cos_encoded = torch.cos(2 * np.pi * data / max_val)
    return sin_encoded, cos_encoded

def encode_timestamp(timestamp):
    month = timestamp.month
    hour = timestamp.hour
    weekday = timestamp.weekday()

    month_sin, month_cos = cyclical_encode(torch.tensor(month), 12)
    hour_sin, hour_cos = cyclical_encode(torch.tensor(hour), 24)
    weekday_sin, weekday_cos = cyclical_encode(torch.tensor(weekday), 7)

    return torch.stack((month_sin, month_cos, hour_sin, hour_cos, weekday_sin, weekday_cos))

In [ ]:
for elem in processed_trajectories:
    user_id, trajectory_edges, transport_mode, timestamp = elem
    # Encode user_id
    elem[0] = torch.tensor([user_id])
    # Encode transport mode
    elem[2] = torch.tensor([encode_transport_mode(transport_mode, transport_mode_dict)])

    # Encode timestamp
    elem[3] = encode_timestamp(timestamp)


## Final step: creating the dataset
Finally, we will have to split our data into train, validation and test dataset. We will use 80% of the data for training 10% for validation and 10% for testing.

In [ ]:
# First shuffle the data
processed_trajectories = np.random.shuffle(processed_trajectories)

In [ ]:
# split data into train, validation and test (0.8,0.1,0.1)
train_data = processed_trajectories[:int(0.8 * len(processed_trajectories))]
val_data = processed_trajectories[int(0.8 * len(processed_trajectories)):int(0.9 * len(processed_trajectories))]
test_data = processed_trajectories[int(0.9 * len(processed_trajectories)):]

# save the data
train_path = 'dataset/train_data.pkl'
val_path = 'dataset/val_data.pkl'
test_path = 'dataset/test_data.pkl'

with open(train_path, 'wb') as file:
    pickle.dump(train_data, file)

with open(val_path, 'wb') as file:
    pickle.dump(val_data, file)

with open(test_path, 'wb') as file:
    pickle.dump(test_data, file)

# Done!